# Calculate extreme indices

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import dask.diagnostics # dask allows you to check how long something is taking to load
import climtas # needed to count event statistics with a specified duration
import seaborn as sns
import scipy.stats # to calculate correlation

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR
import Extreme_indices_functions as funcX
# import obs_functions as funcO

In [3]:
# define path where obs data is store
path = '/g/data/w48/kb6999/Observations/obs_netcdfs_T/'

In [4]:
# open daily T data from 5 stations 
daily_T = xr.open_dataset(f'{path}Daily_T_Aus_5S.nc')
daily_T

<xarray.Dataset>
Dimensions:  (station: 5, time: 15705)
Coordinates:
  * time     (time) datetime64[ns] 1878-01-01 1878-01-02 ... 1920-12-31
  * station  (station) object 'Adelaide' 'Cape_Otway' ... 'Melbourne' 'Sydney'
Data variables:
    Tmin     (station, time) float64 ...
    Tmax     (station, time) float64 ...

Here are the periods of data around the Krakatoa eruption that seem to be homogenous, based on analysis of the monthly records. There are no breakpoints near Krakatoa (with the possible exception of Adelaide), so they shouldn't affect us. But we need to acknowledge that there may be remaining quality issues with the observations, just as there are with the reanalysis! The joys of working with the real world!
 
Adelaide: Tmax and Tmin [1872-1887] - clim [1872-1887]  less than 30 years

Cape Otway: Tmax [1864-1898], Tmin [1867-1888] - overall [1867-1888] less than 30 years

Eversleigh: Tmax and Tmin [1877-1922] - clim [1880-1910 or 1890-1920]

Melbourne: Tmax [1855-1941], Tmin [1870-1908] - overall [1870-1908] - clim [1870-1900]

Sydney: Tmax [1869-1912], Tmin [1866-1919] - overall [1869-1912] - clim [1870-1900 or 1880-1910]

Overall good period: 1877-1888 (ten years around Krakatoa)

In [13]:
# years to take as base period for percentiles
start_dates = [1872, 1867, 1880, 1870, 1870]
end_dates = [1887, 1888, 1910, 1900, 1900]

In [23]:
#calculate the length of each base period
for d, dates in enumerate(end_dates):
    period = end_dates[d] - start_dates[d]
    print(period)

15
21
30
30
30


In [21]:
ds=[]
# calculate extreme indices for each station 
for s, station in enumerate(daily_T.station):
    ex = funcX.extreme_indices(daily_T.sel(station=station), 'M', f'{start_dates[s]}', f'{end_dates[s]}')
    ds.append(ex)

In [22]:
ex_indices = xr.concat(ds, dim='station', coords='minimal')
ex_indices

<xarray.Dataset>
Dimensions:  (station: 5, time: 516)
Coordinates:
  * time     (time) datetime64[ns] 1878-01-31 1878-02-28 ... 1920-12-31
  * station  (station) <U10 'Adelaide' 'Cape_Otway' ... 'Melbourne' 'Sydney'
Data variables: (12/14)
    FD       (station, time) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    SU       (station, time) int64 31 21 14 8 0 0 0 1 2 ... 4 0 0 0 0 1 6 14 16
    ID       (station, time) int64 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0
    TR       (station, time) int64 7 6 7 0 0 0 0 0 0 1 4 ... 0 0 0 0 0 0 0 0 1 4
    TXx      (station, time) float64 45.2 36.7 39.6 31.8 ... 26.0 27.9 36.7 33.3
    TNx      (station, time) float64 28.6 25.9 26.8 19.2 ... 15.6 16.7 20.9 21.1
    ...       ...
    TN10p    (station, time) float64 0.0 0.0 0.0 0.0 0.0 ... 6.667 0.0 0.0 0.0
    TX10p    (station, time) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    TN90p    (station, time) float64 38.71 32.14 35.48 10.0 ... 0.0 6.667 30.0
    TX90p    (station, time) float64 58.06 25.0 16.13 0.0 ... 6.452 36.67 36.67
    DTR      (station, time) float64 16.49 11.43 9.635 ... 8.587 9.387 7.557
    ETR      (station, time) float64 34.5 27.1 29.3 23.0 ... 19.3 18.6 24.0 19.6

In [8]:
# save as netcdf
ex_indices.to_netcdf(f'{path}Obs_extreme_indices.nc')

In [9]:
# open netcdf of extreme indices
ex_index = xr.open_dataset(f'{path}Obs_extreme_indices.nc')
ex_index

<xarray.Dataset>
Dimensions:  (station: 5, time: 516)
Coordinates:
  * time     (time) datetime64[ns] 1878-01-31 1878-02-28 ... 1920-12-31
  * station  (station) object 'Adelaide' 'Cape_Otway' ... 'Melbourne' 'Sydney'
Data variables: (12/14)
    FD       (station, time) int64 ...
    SU       (station, time) int64 ...
    ID       (station, time) int64 ...
    TR       (station, time) int64 ...
    TXx      (station, time) float64 ...
    TNx      (station, time) float64 ...
    ...       ...
    TN10p    (station, time) float64 ...
    TX10p    (station, time) float64 ...
    TN90p    (station, time) float64 ...
    TX90p    (station, time) float64 ...
    DTR      (station, time) float64 ...
    ETR      (station, time) float64 ...